# __TensorFlow - Build a Restricted Boltzmann Machine__
---
</br>

## Intro into RBM 简单介绍
- RBM可以用于对输入数据的**Encoding**工作，它可以将输入数据在无监督的方式下转化为一组用于描述该输入的特征向量，并将其用于后续任务学习
- RBM针对输入产生的特征向量输出，经由**Decode**步骤后应可被还原为原输出，这样保证我们对RBM模型学习结果的可控，并且一定程度上保证模型可以提取出能够有效特征来描述输入数据
- 也就是说，RBM是无监督的，全自动的，对一类学习任务的输入数据进行特征向量提取的专用模型，相比于最简单的BoWs或者One-Hot coding策略，RBM有更大的可能性，以非线性的方式提取到输入数据的实际有意义特征
- RBM专注在良好的去根据输入数据原本分布方式进行拟合，是生成式模型，通过已学习到的输入样本特征分布状况，对新进入的样本根据已知分布进行映射，从而得到其特征向量描述

## RBM usages 本模型能够用于
- 协同过滤
- 数据降维
- 特征提取
- 分类
- 构建如`Knowledge Graph`类似模型时，对Entity进行自动Encoding
- 话题模型



In [2]:
import tensorflow as tf
import numpy as np 
from tensorflow.examples.tutorials.mnist import input_data

VIS_DIM = 784 # 示例采用MNIST数据集，每张图片有784个Pixels
HID_DIM = 666
ALPHA = .1
EPOCH = 20
BATCH = 100

mnist = input_data.read_data_sets('../dataset/Mnist/MNIST/raw', one_hot=True) # use one-hot coding to encode the mnist dataset
train_X = mnist.train.images
train_Y = mnist.train.labels
test_X = mnist.test.images
test_Y = mnist.test.labels

Extracting ../dataset/Mnist/MNIST/raw\train-images-idx3-ubyte.gz
Extracting ../dataset/Mnist/MNIST/raw\train-labels-idx1-ubyte.gz
Extracting ../dataset/Mnist/MNIST/raw\t10k-images-idx3-ubyte.gz
Extracting ../dataset/Mnist/MNIST/raw\t10k-labels-idx1-ubyte.gz


## RBM structure 网络结构简介
- RBM模型结构非常简洁明了，直接由`visible`与`hidden`两层构成
- `visible`层根据输入样本的维度进行定义，`hidden`层决定RBM模型输出特征向量维度
  - 这代表我们的输入样本将经过一个FC层，将其维度进行一次映射更改
  - 这也意味着我们需要两组`bias`值，第一个作用于正向Encoder的偏移量，第二个作用于反向Decoder的偏移量
- 每个`visible`的单元将经过一次激活函数，并且给出一个概率值，作为本神经元所得会被之后采样选中的概率
- 正因如此，我们需要计算两段梯度，即`positive`和`negative`两个方向(原输入->网络输出；还原输入->网络输出)
  - 在还原操作时，我们直接利用矩阵转置相乘作为还原方法进行Decoder训练

$Encoder: Sample(Sigmoid(Input_t \times FC_{Weight} + Bias_{hidden})) = Hidden_t$

$Docoder: Sample(Sigmoid(HiddenVec_t \times {FC_{Weight}}^T + Bias_{visible})) = RestoreVis_t$

- `positive`方向可以看做是从原始输入到特征自动提取向量的信息变化度量
- `negative`方向可以看作是从所得还原数据再次到特征提取结果的信息变化量
- 使用输入，输出两者向量的外积作为度量方式：两者外积所得矩阵与Weight矩阵维度相同
- 两者越是接近，代表得到的特征提取向量越能够良好的表现出我们的输入数据原本特征

$positive: positive_{grad} = {Visible_{t}}^T \times HiddenVec_{t}$

$negative: negative_{grad} = {RestoreVis_{t}}^T \times HiddenVec_{t+1}$

${FC_{Weight}}_{New} = FC_{Weight} + \alpha \times \frac{(positive_{grad} - negative_{grad})}{|Sample|}$

In [ ]:
def sample_prob(hidden_prob): # 根据提供的单元概率执行采样
    return tf.nn.relu(tf.sign(hidden_prob - tf.random_uniform(tf.shape(hidden_prob))))

input_x = tf.placeholder(tf.float32, [None, VIS_DIM]) # 输入数据维度
rbm_weight = tf.placeholder(tf.float32, [VIS_DIM, HID_DIM]) # FC层参数维度
rbm_hidden_bias = tf.placeholder(tf.float32, [HID_DIM])
rbm_visible_bias = tf.placeholder(tf.float32, [VIS_DIM])

hidden_prob = tf.nn.sigmoid(tf.matmul(input_x, rbm_weight) + rbm_hidden_bias) # 利用Sigmoid激活得到第一次正向传播各单元概率
hidden_sample = sample_prob(hidden_prob) # 获取第一次传播的采样结果
restore_input = tf.nn.sigmoid(tf.matmul(hidden_prob, tf.transpose(rbm_weight)) + rbm_visible_bias) # 各单元还原概率，此处直接使用了上一处sample前所得的prob进行还原，也为一种常用方法
resinput_hidden = tf.nn.sigmoid(tf.matmul(restore_input, rbm_weight) + rbm_hidden_bias) # 再次求取特征映射结果

positive_grad = tf.matmul(tf.transpose(input_x), hidden_sample)
negative_grad = tf.matmul(tf.transpose(restore_input), resinput_hidden)
sample_size = tf.cast(tf.shape(input_x)[0], dtype=tf.float32) # 输入的学习样本数量，即Batch大小
# 由此可得，我们的更新量Δ应该为
delta = (positive_grad - negative_grad) / sample_size

# 利用定义好的α值决定更新比率，即得：
rbm_weight_new = rbm_weight + ALPHA * delta
rbm_visible_bias_new = rbm_visible_bias + ALPHA * tf.reduce_mean(input_x - restore_input, 0)
rbm_hidden_bias_new = rbm_hidden_bias + ALPHA * tf.reduce_mean(hidden_prob - resinput_hidden, 0)

# 我们期待两者能够接近，也就是最小化error
error = tf.reduce_mean(tf.square(input_x - restore_input)) 

## 简单RBM批量训练示例
- 我们需要为`visible`，`hidden`和`weight`初始化
- 由于我们需要进行反复两次RBM前向传播以计算error和优化三个参数矩阵，我们需要对上一时刻的三个参数矩阵进行保存
- 根据设定的`BATCH`和`EPOCH`,调起`tf.Session()`进行模型训练
- 我们根据`mean=0, std=.01`的`正态分布`为RBM模型的初始`weight`矩阵随机生成参数

In [ ]:
MEAN = .0
STD = .01
# 参数初始化，定义维度
cur_rbm_weight = np.zeros([VIS_DIM, HID_DIM], np.float32)
cur_rbm_visible = np.zeros([VIS_DIM], np.float32)
cur_rbm_hidden = np.zeros([HID_DIM], np.float32)
# 我们将使用sav批次的参数作为最初传入的初始模型参数
sav_rbm_weight = np.random.normal(MEAN, STD, [VIS_DIM, HID_DIM])
sav_rbm_visible = np.zeros([VIS_DIM], np.float32)
sav_rbm_hidden = np.zeros([HID_DIM], np.float32)

# 唤起Session
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    for epoch in range(EPOCH):
        for batch_start, batch_end in zip(range(0, len(train_X), BATCH), range(BATCH, len(train_X), BATCH)):
            cur_batch = train_X[batch_start: batch_end]
            # 将参数传入，求取更新后的三参数矩阵
            cur_rbm_weight, cur_rbm_visible, cur_rbm_hidden = sess.run([rbm_weight_new, rbm_visible_bias_new, rbm_hidden_bias_new], feed_dict={input_x: cur_batch, rbm_weight: sav_rbm_weight, rbm_visible_bias: sav_rbm_visible, rbm_hidden_bias: sav_rbm_hidden})
            # 将结果保存/更新
            sav_rbm_weight = cur_rbm_weight
            sav_rbm_visible = cur_rbm_visible
            sav_rbm_hidden = cur_rbm_hidden
        # 开始下一次传播，此次用于计算error
        cur_error = sess.run(error, feed_dict={input_x: cur_batch, rbm_weight: sav_rbm_weight, rbm_visible_bias: sav_rbm_visible, rbm_hidden_bias: sav_rbm_hidden})
        print("Now at epoch:", epoch + 1, "and the current reconstrucion error is:", cur_error)

## 构建一个更加完善的RBM网络
- 采用`Xavier`方式初始化`weight`层参数
- 将网络整体封装入类中
- 利用贝努力分布构建BBRBM
- 利用高斯分布构建GBRMB

### Xavier参数构建
- 通过保证各层输出的方差尽可能相似来辅助梯度流动
- 一定程度上减少计算开销，由此加快模型收敛
- 默认只考虑输入维度，即：
$Var(weight_{i}) = \frac{1}{dim_{input}}$
- 此处目标为使`weight`分布符合(此时考虑输入输出两层维度)：
$U[-\frac{\sqrt{6}}{\sqrt{dim_{input}+dim_{out}}}, \frac{\sqrt{6}}{\sqrt{dim_{input}+dim_{out}}}]$

In [ ]:
import tensorflow as tf
import numpy as np 

# 定义Xavier采样方法
def xavier_weight_init(dim_in, dim_out, xavier_const=1., dtype=np.float32):
    base = np.sqrt(6.0 / (dim_in + dim_out)) * xavier_const
    return tf.random_uniform((dim_in, dim_out), minval=-base, maxval=base, dtype=dtype)

# 定义RMB类
class RBM():

    def __init__(self, dim_visible, dim_hidden, xavier_const=1., lr=.01):
        # 将参数保存入网络中
        self.dim_visible = dim_visible
        self.dim_hidden = dim_hidden
        self.xavier_const = xavier_const
        self.lr = lr 
        # 定义输入变量空间
        self.input_x = tf.placeholder(dtype=tf.float32, shape=[None, self.dim_visible])
        # 定义、初始化参数矩阵维度
        self.rbm_weight = tf.Variable(xavier_weight_init(dim_in=self.dim_visible, dim_out=self.dim_hidden, xavier_const=self.xavier_const), dtype=tf.float32)
        self.rbm_visible = tf.Variable(tf.zeros([self.dim_visible]), dtype=tf.float32)
        self.rbm_hidden = tf.Variable(tf.zeros([self.dim_hidden]), dtype=tf.float32)
        # 存储更新的三个参数矩阵与变化量，它们在不同采样的模型中表现不同，我们在init时仅暂时进行声明，其计算定义交由forward函数解决。[weight, visible, hidden]
        self.rbm_weights_new = None
        # 进行RBM前向传播定义
        self.cur_hidden = None
        self.cur_reconstruction = None
        # 定义各类参数计算
        self.forward()
        # error计算
        self.error = tf.reduce_mean(tf.square(self.input_x - self.cur_reconstruction))
        # 获取tf.Session()
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
    
    def forward(self):
        # 计算RMB传播，将更改量记入
        hidden_prob = tf.nn.sigmoid(tf.matmul(self.input_x, self.rbm_weight) + self.rbm_hidden)
        hidden_sample = self.sample_prob(hidden_prob)
        reconstruction = tf.nn.sigmoid(tf.matmul(hidden_prob, tf.transpose(self.rbm_weight)) + self.rbm_visible)
        resinput_hidden = tf.nn.sigmoid(tf.matmul(reconstruction, self.rbm_weight) + self.rbm_hidden)
        # 计算positive与negative梯度
        positive_grad = tf.matmul(tf.transpose(self.input_x), hidden_sample)
        negative_grad = tf.matmul(tf.transpose(reconstruction), resinput_hidden)
        # 进行参数优化
        self._forward(positive_grad, negative_grad, reconstruction, hidden_prob, resinput_hidden)
        # 完成RBM向前传播计算
        self.cur_hidden = tf.nn.sigmoid(tf.matmul(self.input_x, self.rbm_weight) + self.rbm_hidden)
        self.cur_reconstruction = tf.nn.sigmoid(tf.matmul(self.cur_hidden, tf.transpose(self.rbm_weight)) + self.rbm_visible)
    
    # 用于辅助forward计算，将各个RBM中相同的部分：参数优化部分提出，避免过多重复代码
    def _forward(self, positive_grad, negative_grad, reconstruction, hid_sample, rec_hidden):
        self.rbm_weight = self.rbm_weight + self.lr * ((positive_grad - negative_grad) / tf.cast(tf.shape(self.input_x)[0], dtype=tf.float32))
        self.rbm_visible = self.rbm_visible + self.lr * tf.reduce_mean(self.input_x - reconstruction, 0)
        self.rbm_hidden = self.rbm_hidden + self.lr * tf.reduce_mean(hid_sample - rec_hidden, 0)      
        self.rbm_weights_new = [self.rbm_weight, self.rbm_visible, self.rbm_hidden]

    def sample_prob(self, hidden_prob):
        return tf.nn.relu(tf.sign(hidden_prob - tf.random_uniform(tf.shape(hidden_prob))))

    def partial_fit(self, batch_x):
        self.sess.run(self.rbm_weights_new, feed_dict={self.input_x: batch_x})

    def test(self, test_x):
        return self.sess.run(self.error, feed_dict={self.input_x: test_x})
    
    def fit(self, train_x, epoch=20, batch=100):
        data_size = train_x.shape[0]
        error_list = []
        for _epoch in range(epoch):
            for batch_start, batch_end in zip(range(0, data_size, batch), range(batch, data_size, batch)):
                batch_data = train_x[batch_start: batch_end]
                # 将数据传入训练
                self.partial_fit(batch_data)
                if batch_start == 0:
                    cur_error = self.test(batch_data)
                    print("The reconstruction error on the 1st batch sample of epoch", _epoch, "is:", cur_error)
                    error_list.append(cur_error)
        return error_list

    # 调用本方法得到从hidden层的重构图片，可以用于可视化检测
    def reconstruct(self, input_img):
        return self.sess.run(self.cur_reconstruction, feed_dict={self.input_x: input_img})
    
    def save_weights(self, filename, rbmname):
        saver = tf.train.Saver({rbmname + '_weight': self.rbm_weight, rbmname + '_visible': self.rbm_visible, rbmname + '_hidden': self.rbm_hidden})
        return saver.save(self.sess, filename)
    
    def load_weights(self, filename, rbmname):
        saver = tf.train.Saver({rbmname + '_weight': self.rbm_weight, rbmname + '_visible': self.rbm_visible, rbmname + '_hidden': self.rbm_hidden})
        saver.restore(self.sess, filename)

### 使用构建的RBM
- 此处我们仍旧使用`MNIST`数据集
- 利用`reconstruction`返回一站图片的重构结果，进行图像对比

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

VIS_DIM = 784
HID_DIM = 196
LR = .1
EPOCH = 10
BATCH = 30
IMAGE_ID = 233 

mnist = input_data.read_data_sets('../dataset/Mnist/MNIST/raw', one_hot=True)
train_X = mnist.train.images
test_X = mnist.test.images

simpleRBM = RBM(dim_visible=VIS_DIM, dim_hidden=HID_DIM, lr=LR)
training_error_list = simpleRBM.fit(train_X, epoch=EPOCH, batch=BATCH)
final_test_error = simpleRBM.test(test_X)
# for i in range(EPOCH):
#     print("At epoch", i, "the reconstruction error on the 1st batch sample is:", training_error_list[i])
print("Finally the error on the test set is:", final_test_error)

### 查看训练结果

In [ ]:
import matplotlib.pyplot as plt

def show_digit(x):
    plt.imshow(x.reshape((28, 28)), cmap=plt.cm.gray)
    plt.show()

image = train_X[IMAGE_ID]
image_rec = simpleRBM.reconstruct(image.reshape(1,-1))

show_digit(image)
show_digit(image_rec)

## 构建基于贝努力分布采样的RBM (`Bernoulli-Bernoulli RBM`)
- 针对服从贝努力分布的`binary`类型变量进行分布学习
- 为简单的两点分布 (0-1分布): 
$P(x=1)=n$; $P(x=0)=1-n$

In [ ]:
class BBRBM(RBM):

    def __init__(self, *args, **kwargs):
        RBM.__init__(self, *args, **kwargs)

    # 仅重写不同部分，此时我们将更改采样方法
    def forward(self):
        # 计算RMB传播，将更改量记入
        hidden_prob = tf.nn.sigmoid(tf.matmul(self.input_x, self.rbm_weight) + self.rbm_hidden)
        # 使用贝努力分布构建采样
        hidden_sample = self.sample_bernoulli(hidden_prob)
        reconstruction = tf.nn.sigmoid(tf.matmul(hidden_sample, tf.transpose(self.rbm_weight)) + self.rbm_visible)
        recon_hidden = tf.nn.sigmoid(tf.matmul(reconstruction, self.rbm_weight) + self.rbm_hidden)
        # 计算positive与negative梯度
        positive_grad = tf.matmul(tf.transpose(self.input_x), hidden_prob)
        negative_grad = tf.matmul(tf.transpose(reconstruction), recon_hidden)
        # 进行参数优化
        self._forward(positive_grad, negative_grad, reconstruction, hidden_prob, recon_hidden)
        # 完成RBM向前传播计算
        self.cur_hidden = tf.nn.sigmoid(tf.matmul(self.input_x, self.rbm_weight) + self.rbm_hidden)
        self.cur_reconstruction = tf.nn.sigmoid(tf.matmul(self.cur_hidden, tf.transpose(self.rbm_weight)) + self.rbm_visible) 

    def sample_bernoulli(self, hidden_prob):
        return tf.nn.relu(tf.sign(hidden_prob - tf.random_uniform(tf.shape(hidden_prob))))

### 使用BBRBM
- 同样，采用了`MNIST`数据进行特征提取
- 取出一张图片直接进行比对

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

VIS_DIM = 784
HID_DIM = 64
LR = .1
EPOCH = 30
BATCH = 10
IMAGE_ID = 233 

mnist = input_data.read_data_sets('../dataset/Mnist/MNIST/raw', one_hot=True)
train_X = mnist.train.images
test_X = mnist.test.images

simpleBBRBM = BBRBM(dim_visible=VIS_DIM, dim_hidden=HID_DIM, lr=LR)
training_error_list = simpleBBRBM.fit(train_X, epoch=EPOCH, batch=BATCH)
final_test_error = simpleBBRBM.test(test_X)
# for i in range(EPOCH):
#     print("At epoch", i, "the reconstruction error on the 1st batch sample is:", training_error_list[i])
print("Finally the error on the test set is:", final_test_error)

### 查看训练结果

In [ ]:
image = train_X[IMAGE_ID]
image_rec = simpleBBRBM.reconstruct(image.reshape(1,-1))

show_digit(image)
show_digit(image_rec)

## 构建GBRBM
- 与BBRBM类似，我们只需要实现其与RBM中不同的采样部分
- 我们需要去更改我们的Sample方法

In [ ]:
class GBRBM(RBM):
    
    def __init__(self, sigma=1., *args, **kwargs):
        self.sigma = sigma
        RBM.__init__(self, *args, **kwargs)

    def sample_gaussian(self, x, sigma):
        return x + tf.random_normal(tf.shape(x), mean=0.0, stddev=sigma, dtype=tf.float32)
    
    def sample_bernoulli(self, hidden_prob):
        return tf.nn.relu(tf.sign(hidden_prob - tf.random_uniform(tf.shape(hidden_prob))))
    
    def forward(self):
        # 计算RMB传播，将更改量记入
        hidden_prob = tf.nn.sigmoid(tf.matmul(self.input_x, self.rbm_weight) + self.rbm_hidden)
        reconstruction = self.sample_gaussian(tf.nn.sigmoid(tf.matmul(self.sample_bernoulli(hidden_prob), tf.transpose(self.rbm_weight)) + self.rbm_visible), self.sigma)
        recon_hidden = tf.nn.sigmoid(tf.matmul(reconstruction, self.rbm_weight) + self.rbm_hidden)
        # 计算positive与negative梯度
        positive_grad = tf.matmul(tf.transpose(self.input_x), hidden_prob)
        negative_grad = tf.matmul(tf.transpose(reconstruction), recon_hidden)
        # 进行参数优化
        self._forward(positive_grad, negative_grad, reconstruction, hidden_prob, recon_hidden)
        # 完成RBM向前传播计算
        self.cur_hidden = tf.nn.sigmoid(tf.matmul(self.input_x, self.rbm_weight) + self.rbm_hidden)
        self.cur_reconstruction = tf.nn.sigmoid(tf.matmul(self.cur_hidden, tf.transpose(self.rbm_weight)) + self.rbm_visible)

### 使用GBRBM

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

VIS_DIM = 784
HID_DIM = 64
LR = .1
EPOCH = 10
BATCH = 10
IMAGE_ID = 233 

mnist = input_data.read_data_sets('../dataset/Mnist/MNIST/raw', one_hot=True)
train_X = mnist.train.images
test_X = mnist.test.images

simpleGBRBM = GBRBM(dim_visible=VIS_DIM, dim_hidden=HID_DIM, lr=LR)
training_error_list = simpleGBRBM.fit(train_X, epoch=EPOCH, batch=BATCH)
final_test_error = simpleGBRBM.test(test_X)
# for i in range(EPOCH):
#     print("At epoch", i, "the reconstruction error on the 1st batch sample is:", training_error_list[i])
print("Finally the error on the test set is:", final_test_error)

### 查看训练结果

In [ ]:
image = train_X[IMAGE_ID]
image_rec = simpleGBRBM.reconstruct(image.reshape(1,-1))

show_digit(image)
show_digit(image_rec)